In [ ]:
# Simple VTK offscreen render test (inline display)
import vtk
from IPython.display import Image, display

# Create a simple geometry (sphere)
sphere = vtk.vtkSphereSource()
sphere.SetPhiResolution(30)
sphere.SetThetaResolution(30)
sphere.Update()

# Mapper and actor
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(sphere.GetOutputPort())
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Renderer and offscreen render window
ren = vtk.vtkRenderer()
ren.AddActor(actor)
ren.SetBackground(1.0, 1.0, 1.0)

renWin = vtk.vtkRenderWindow()
renWin.SetOffScreenRendering(1)
renWin.AddRenderer(ren)
renWin.SetSize(600, 400)
renWin.Render()

# Capture window to image
w2i = vtk.vtkWindowToImageFilter()
w2i.SetInput(renWin)
w2i.Update()

# Write PNG to memory and display inline
writer = vtk.vtkPNGWriter()
writer.SetWriteToMemory(1)
writer.SetInputConnection(w2i.GetOutputPort())
writer.Write()

png_data = writer.GetResult()
display(Image(data=bytes(memoryview(png_data))))

print("VTK offscreen render OK")


In [ ]:
# Tutorial 1 (Hello Plant) — Setup
# Use package data paths; avoid depending on repo visualisation helpers
import os

from IPython.display import Image, display

import plantbox as pb


In [ ]:
# Load parameters from packaged data
plant = pb.Plant()
param_path = os.path.join(pb.data_path(), "structural", "plant", "fspm2023.xml")
plant.readParameters(param_path)
plant.initialize()

# Simulate a small time to keep it light
plant.simulate(20)


In [ ]:
# Export results to files
os.makedirs("results", exist_ok=True)
plant.write("results/example_plant.vtp")
ana = pb.SegmentAnalyser(plant)
ana.write("results/example_plant_segs.vtp")
print("Wrote results/example_plant.vtp and results/example_plant_segs.vtp")


In [ ]:
import os

import plantbox as pb  # |\label{l13:cplantbox}|
import vtk_plot as vp  # type: ignore

plant = pb.Plant()  # Create a new plant |\label{l13:plant}|

# Open plant and root parameter from a file using packaged data paths
path = os.path.join(pb.data_path(), "structural", "plant") + "/"
name = "fspm2023"
plant.readParameters(path + name + ".xml")  # |\label{l13:readparameters}|

plant.initialize()  # Initialize |\label{l13:initialize}|

simtime = 40  # days
plant.simulate(simtime)  # Simulate|\label{l13:simulate}|

os.makedirs("results", exist_ok=True)
# Export final result (as vtp)
plant.write("results/example_plant.vtp")  # using polylines |\label{l13:write_poly}|

ana = pb.SegmentAnalyser(plant)
ana.write("results/example_plant_segs.vtp")  # using segments |\label{l13:write_segs}|

vp.plot_plant(plant, "age")  # e.g. organType, subType, age |\label{l13:plot_plant}|


Notes:
- This notebook mirrors tutorial 1 (Hello Plant) using `plantbox.data_path()` for robust data access.
- VTK plotting runs offscreen; ensure the kernel uses the venv where `vtk` is installed.
- Files are written into `results/` under the repo root.
